In [2]:
import sys
import os
sys.path.append(os.path.abspath("/content/drive/MyDrive/Project"))

In [3]:
!pip install tensorflow_text
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 46.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=b5305d7b43929c1fbb43e2a176a3e893bf5246f23f0253b4a9dd6c3d567bc062
  Stored in directory: 

In [4]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def classify_sentence(sentence, model):
    sentence = np.array([[sentence]])
    print(sentence)
    y_predict = model.predict(sentence)
    y_predict = y_predict.flatten()
    y_predict = np.where(y_predict > 0.5, 1, 0)

    return y_predict[0]

def prepare_data():
  import pandas as pd
  final_dataset = pd.read_csv("/content/drive/MyDrive/Project/Processed_Data/fin.csv")
  X = final_dataset["Comments"]
  y = final_dataset["Type"]
  final_dataset.head()

  final_dataset.groupby('Type').describe()
  final_dataset["Type"].value_counts()

    #df_roast = final_dataset[final_dataset["Type"] == 0]
  df_toast = final_dataset[final_dataset["Type"] == 1]
    #df_roast_downsampled = df_roast.sample(df_toast.shape[0])

  sentences = []
  for i in df_toast["Comments"] :
    sentences.append(i)

  return sentences

def sentence_vec(sentences):
  model_name = 'bert-base-nli-mean-tokens'
  from sentence_transformers import SentenceTransformer

  md = SentenceTransformer(model_name)

  Sentence_vecs = md.encode(sentences)

  return Sentence_vecs

def get_convert_sentence(str1, Sentence_vecs, sentences):
  model_name = 'bert-base-nli-mean-tokens'
  from sentence_transformers import SentenceTransformer

  md = SentenceTransformer(model_name)

  sen = md.encode(str1)

  from sklearn.metrics.pairwise import cosine_similarity
  result = cosine_similarity([sen], Sentence_vecs[:])

  m = 0
  final_conversion = ""
  for i in range(0, len(result[0])):
    if(m<result[0][i]):
      m = result[0][i]
      final_conversion = sentences[i]

  return final_conversion

In [12]:
def run_app(str1):
  import pickle
  #sentence = input("Enter the sentence: ")
  model = load_model("/content/drive/MyDrive/Project/model")
  result = classify_sentence(str1, model)
  final = ""
  if(result == 1):
    print("Roast Sentence")
    sentences = prepare_data()
    model_name = 'bert-base-nli-mean-tokens'
    #from sentence_transformers import SentenceTransformer

    #md = SentenceTransformer(model_name)
    vec_encodings = []
    #vec_encodings = md.encode(sentences)
    with open("/content/drive/MyDrive/Project/vec_encodings", "rb") as f:vec_encodings = pickle.load(f)
    final_convert = get_convert_sentence(str1, [vec_encodings], sentences)
    #print(final_convert)
    final = final_convert
    return final
  else:
    return "Already Toast!!"

In [7]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,360 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,299 kB]
Get:13 https://ngrok-agent.s3.amazonaws

In [10]:
!pip install flask_ngrok
!pip install pyngrok
!ngrok config add-authtoken 2RZVrnpTreJEGpl3F0guVce60P5_4cmqLGEfNGvuBRLU1fpUt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [13]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)

run_with_ngrok(app)

return_file = [
    { 'sentence': "Whats UP!"}
]


@app.route('/', methods=['GET'])
def get_incomes():
  return jsonify(return_file)


@app.route('/convert', methods=['POST'])
def get_sentence():
    str1 = request.form['sentence']
    str1 = run_app(str1)
    return_file[0]['sentence'] = str1
    return jsonify(return_file)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1a8e-34-134-31-206.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
[['You look fat and ugly']]
1/1 [==============================] - 4s 4s/step
Roast Sentence


INFO:werkzeug:127.0.0.1 - - [23/Jun/2023 05:34:25] "POST /convert HTTP/1.1" 200 -


In [ ]:
#import pickle
#with open("vec_encodings", "wb") as f:
  #pickle.dump(vec_encodings, f)

In [ ]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!flask --app drive/MyDrive/Project/App.py run


2023-06-22 12:05:41.873549: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
 * Ignoring a call to 'app.run()' that would block the current 'flask' CLI command.
   Only call 'app.run()' in an 'if __name__ == "__main__"' guard.
 * Serving Flask app 'drive/MyDrive/Project/App.py'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
^C
